In [15]:
import pandas as pd
import nltk
from nltk import SnowballStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/sofia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
data = pd.read_csv("../data/interim/dataset_v2.csv")

In [17]:
data.head()

,Unnamed: 0,manually_assigned_tag,url,content
0,0,supermarket,https://www.walmart.com.ar,nueva nueva correctamente puede cerrar lacteos...
1,0,supermarket,https://www.makro.com.ar,logo entra cuenta quiero fornecedor sobre quie...
2,0,supermarket,https://www.alvearsupermercados.com.ar/,alvearsupermercados logo blanco alvearsupermer...
3,0,supermarket,https://www.yaguar.com.ar/,supermercado blanca chaco cordoba cordoba cont...
4,0,supermarket,https://www.libertadsa.com.ar/,logo separador cordoba separador separador sep...


In [18]:
data = data.drop(columns=['Unnamed: 0'])

In [19]:
data.columns

Index(['manually_assigned_tag', 'url', 'content'], dtype='object')

In [20]:
data.head()

,manually_assigned_tag,url,content
0,supermarket,https://www.walmart.com.ar,nueva nueva correctamente puede cerrar lacteos...
1,supermarket,https://www.makro.com.ar,logo entra cuenta quiero fornecedor sobre quie...
2,supermarket,https://www.alvearsupermercados.com.ar/,alvearsupermercados logo blanco alvearsupermer...
3,supermarket,https://www.yaguar.com.ar/,supermercado blanca chaco cordoba cordoba cont...
4,supermarket,https://www.libertadsa.com.ar/,logo separador cordoba separador separador sep...


In [21]:
%%writefile Stemmer.py
import nltk
from collections import defaultdict
from nltk import SnowballStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
class Stemmer(): # naive class
    def __init__(self):
        self.stems = list()
        self.stemmed_freqs = defaultdict(list)
        self.tokens = list()

    def tokenize(self, document):
        '''
        tokenize document and remove (some of) non-spanish words
        '''
        if not isinstance(document, list):
            if isinstance(document, str):
                self.tokens = document.split(' ')
            else:
                raise TypeError('str or dict expected, {} found.'.format(type(document)))
        else:
            self.tokens = document
    
    def stemm(self, document):
        '''
        document stemming
        '''
        if len(self.tokens) == 0: #empty list
            self.tokenize(document)
        else:
            spanishstemmer = SnowballStemmer('spanish')
            self.stems = [spanishstemmer.stem(token) for token in self.tokens] 

Overwriting Stemmer.py


In [22]:
# Delete rows with empty 'content' field
data.dropna(subset = ["content"], inplace=True)

In [23]:
def remove_short_words(stringObj, wordLong = 4):
    '''
    remove words with lenght <= wordLong
    
    return: list
    '''
    listObj = stringObj.split(' ')
    words = list()
    words = [word for word in listObj if len(word)>wordLong]
#     if len(stringObj) > wordLong:
#         print(stringObj)
#         words.append(stringObj)
    return words

In [24]:
data['content'] = data['content'].map(lambda x: remove_short_words(x))

In [25]:
data.to_csv('../data/processed/processed_dataset.csv')

In [26]:
def remove_less_frequent_words(myList, frequency=10):
    from collections import Counter
    words_freq = Counter(myList)
    result = list()
    words = [[elem[0]]*elem[1] for elem in words_freq.items() if elem[1] >= frequency]    
    for val in words:
        result.extend(val)
    return result    

In [27]:
data['content'] = data['content'].map(lambda x: remove_less_frequent_words(x, 2))

In [28]:
data.to_csv('../data/processed/processed_dataset_less_frequent_words_removed.csv')


#dejaste leyendo dataset_v2